In [ ]:
!pip install -U torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


self.rnn을 RNN, GRU, LSTM으로 변경해 줄 수 있습니다.
함수 사용 방법은 아래 링크를 참고해주세요<br>
GRU : https://pytorch.org/docs/stable/generated/torch.nn.GRU.html<br>
LSTM : https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchtext.legacy import data, datasets # 라이브러리 에러로 인해 구버전의 torchtext.legacy 사용

# 하이퍼파라미터 정의
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 20
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("다음 기기로 학습합니다:", DEVICE)
## colab GPU 사용 방법 --> 상단 메뉴에서 "런타임"클릭, "런타임 유형 변경"클릭, 하드웨어 가속기를 "GPU"로 변경 ##

# 데이터 로딩하기
print("데이터 로딩중...")
TEXT = data.Field(sequential=True, batch_first=True, lower=True) # 데이터셋 -> 텐서 변환 설정 저장
LABEL = data.Field(sequential=False, batch_first=True)
trainset, testset = datasets.IMDB.splits(TEXT, LABEL) # 모델에 입력되는 데이터셋
TEXT.build_vocab(trainset, min_freq=5) # 워드 임베딩을 위한 단어 사전 (최소 등장 = 5번)
LABEL.build_vocab(trainset)

# 학습용 데이터를 학습셋 80% 검증셋 20% 로 나누기
trainset, valset = trainset.split(split_ratio=0.8)
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)
# 아래의 코드로 데이터를 직접 확인해 보세요.
# print(vars(trainset[0]))

vocab_size = len(TEXT.vocab)
n_classes = 2

print("[학습 데이터]: %d [검증 데이터]: %d [테스트 데이터]: %d [단어수]: %d [클래스] %d"
      % (len(trainset),len(valset), len(testset), vocab_size, n_classes))


class RNN(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.5):
        super(RNN, self).__init__()
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        
        # 1. RNN
        # print("Building Basic RNN model...")
        # self.model = nn.RNN(embed_dim, self.hidden_dim, num_layers=self.n_layers, batch_first=True)

        # 2. GRU
        # print("Building Basic GRU model...")
        # self.model = nn.GRU(embed_dim, self.hidden_dim, num_layers=self.n_layers, batch_first=True, dropout=dropout_p)

        # 3. LSTM
        print("Building Basic LSTM model...")
        self.model = nn.LSTM(embed_dim, self.hidden_dim, num_layers=self.n_layers, batch_first=True)

        self.fc1 = nn.Linear(self.hidden_dim, 256)
        self.fc2 = nn.Linear(256, n_classes)

    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0)) # hidden state
        c_0 = self._init_state(batch_size=x.size(0)) # cell state
        # x, _status = self.model(x, h_0)
        x, _status = self.model(x, (h_0, c_0)) # 3. LSTM 

        # 예측을 위해 마지막 output만을 사용
        h_t = x[:,-1,:]
        h_t = F.relu(self.fc1(h_t))
        logit = self.fc2(h_t)  # [b, h] -> [b, o]
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # 레이블 값을 0과 1로 변환
        optimizer.zero_grad()
        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

model = RNN(1, 256, vocab_size, 256, n_classes, 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[에폭: %d] 검증 오차:%5.2f | 검증 정확도:%5.2f" % (e, val_loss, val_accuracy))
    
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

다음 기기로 학습합니다: cuda
데이터 로딩중...
[학습 데이터]: 20000 [검증 데이터]: 5000 [테스트 데이터]: 25000 [단어수]: 46159 [클래스] 2
Building Basic LSTM model...
[에폭: 1] 검증 오차: 0.69 | 검증 정확도:49.84
[에폭: 2] 검증 오차: 0.70 | 검증 정확도:49.08
[에폭: 3] 검증 오차: 0.70 | 검증 정확도:49.46
[에폭: 4] 검증 오차: 0.71 | 검증 정확도:50.20
[에폭: 5] 검증 오차: 0.72 | 검증 정확도:51.68
[에폭: 6] 검증 오차: 0.73 | 검증 정확도:52.18
[에폭: 7] 검증 오차: 0.74 | 검증 정확도:50.86
[에폭: 8] 검증 오차: 0.75 | 검증 정확도:51.62
[에폭: 9] 검증 오차: 0.74 | 검증 정확도:52.04
[에폭: 10] 검증 오차: 0.72 | 검증 정확도:51.94
[에폭: 11] 검증 오차: 0.75 | 검증 정확도:50.58
[에폭: 12] 검증 오차: 0.67 | 검증 정확도:65.50
[에폭: 13] 검증 오차: 0.54 | 검증 정확도:76.90
[에폭: 14] 검증 오차: 0.50 | 검증 정확도:78.84
[에폭: 15] 검증 오차: 0.53 | 검증 정확도:80.02
[에폭: 16] 검증 오차: 0.63 | 검증 정확도:81.18
[에폭: 17] 검증 오차: 0.73 | 검증 정확도:80.00
[에폭: 18] 검증 오차: 0.78 | 검증 정확도:81.56
[에폭: 19] 검증 오차: 0.81 | 검증 정확도:81.00
[에폭: 20] 검증 오차: 0.92 | 검증 정확도:77.68
테스트 오차:  0.61 | 테스트 정확도: 75.52
